In [ ]:
import requests
import json

url = "http://spiderverse-va.byteintl.net/openapi/token"
data = {
    "username": "longlin.kylin@bytedance.com",
    "secret_key": "ff7d8e04587930e2e5766be6377e2ac5294bda0c0fb4c94e91a0c795b01bcfa3",
}
resp = requests.post(url, data)
token = resp.json()["token"]

print(token)

In [ ]:
def parse_url(url):
    if not url.startswith("https://www.youtube.com/"):
        return None
    if "&" in url:
        pruned_url = url.split("&")[0]
    else:
        pruned_url = url
    return pruned_url, pruned_url.split("watch?v=")[1]

In [ ]:
from utils.tos import list_all_objects

api_url = "http://spiderverse-va.byteintl.net/openapi/submit/stream/seed"
headers = {"token": f"{token}", "Content-Type": "application/json"}

annotations_path = "data/annotations/CZ_1_refined.json"
marker = annotations_path.split("/")[-1].split(".")[0].strip("_refined")
target_dir = marker + "/"
with open(annotations_path, "r") as f:
    videos = json.load(f)

round = 0

while True:
    all_data = []
    # Get all existing files
    downloaded_files = list_all_objects(target_dir)

    for video in videos:
        url = video["video_url"]
        pruned_url, video_id = parse_url(url)
        if video_id in downloaded_files:
            continue
        seed_data = {}
        seed_data["source_data"] = video
        seed_data["__custom_args"] = ["-f", "bestvideo[height=720]+bestaudio"]
        seed_data["video_id"] = video_id
        seed_data["store_key"] = video_id + ".mp4"
        seed_data["dir_name"] = marker
        seed_data["url"] = pruned_url
        runtime_vars = {"key": "value"}
        data = {
            "seedSetId": 1614,
            "seedId": 123,
            "data": json.dumps(seed_data),
            "runtimeVars": json.dumps(runtime_vars),
        }
        all_data.append(data)

    if len(all_data) == 0:
        print("All data downloaded.")
        break

    print(f"Round {round}: downloading {len(all_data)} videos.")

    qps = 5
    import time

    # resp = requests.post(api_url, json=all_data[0], headers=headers)
    # print(resp.json())

    for i in range(0, len(all_data), qps):
        batch = all_data[i : i + qps]
        for data in batch:
            resp = requests.post(api_url, json=data, headers=headers)
        if i + qps < len(all_data):  # Don't sleep after the last batch
            time.sleep(5)  # Sleep 1 second between batches to maintain QPS
    
    time.sleep(600)
    round += 1

In [ ]:
url = "http://spiderverse-va.byteintl.net/openapi/submit/stream/seed"
headers = {"token": f"{token}", "Content-Type": "application/json"}

seed_data = {
    # 这个字段我放了你们的数据
    "source_data": {
        "video_id": "CZ_1",
        "video_url": "https://www.youtube.com/watch?v=PnvZZwlN2yk",
        "video_duration": "29:04",
        "video_type": "综艺 - 游戏",
        "qa_list": [
            {
                "question": "Is Stewart Thompson a person who pursues a high-quality life?",
                "answer": "Yes.",
                "question_type": "多线索推理,人物属性建模",
                "knowledge": "",
                "reasoning": "从Stewart讲述自己旅行方式倾向于商务舱可以判断。",
            },
            {
                "question": "What decade of age is J.F. Harris approximately in?",
                "answer": "He is approximately in his forties.",
                "question_type": "多线索推理,人物属性建模",
                "knowledge": "",
                "reasoning": "J.F. Harris提到过自己2003年曾经在芝加哥，后来又说20岁左右在芝加哥，综合推理得知现在J.F. Harris现在大概四十多岁。",
            },
            {
                "question": "Who had a different opinion from the others when voting out the person who was not a millionaire in the first round?",
                "answer": "Aaron.",
                "question_type": "多跳推理,多模态/多语言推理",
                "knowledge": "",
                "reasoning": "首先需要定位到第一轮投票这个片段，再结合视觉听觉对五个人的发言判断。",
            },
            {
                "question": "What economic conditions do the families of children who have a trust fund usually come from?",
                "answer": "The family possesses substantial family wealth.",
                "question_type": "通用知识推理",
                "knowledge": "Tori claimed that she had a trust fund and still worked hard, but she was suspected of not being a millionaire. ",
                "reasoning": "Tori称自己有trust fund，并且拥有自己的事业努力工作，其中一位主持人不相信，表示自己没见过有trust fund的孩子还努力工作的，说明拥有trust fund的孩子的家庭一般都拥有巨大财富。",
            },
            {
                "question": "Is Ralph's a chain grocery store or an individually-operated grocery store? ",
                "answer": "A chain grocery store.",
                "question_type": "通用知识推理",
                "knowledge": "Tori said that she liked going to Ralph's because it could be found everywhere. ",
                "reasoning": "在主持人问Tori平常多久去一次grocery花费多少钱的时候，Tori说自己喜欢去Ralph's，因为它到处都有，配送费很高，说明Ralph's是一家连锁超市。",
            },
        ],
    },
    # 这里必须有，
    "__custom_args": [
        # 必须
        "-f",
        # format的筛选，你们自己确定
        "bestvideo[height=720]+bestaudio",
    ],
    # ytb的视频id
    "video_id": "nhtAkSW8v0M",
    # 存储的文件名
    "store_key": "nhtAkSW8v0M.mp4",
    # 存储的文件夹
    "dir_name": "test_file",
    # ytb视频的链接，格式固定
    "url": "https://www.youtube.com/watch?v=nhtAkSW8v0M",
}

runtime_vars = {"key": "value"}
data = {
    "seedSetId": 1614,
    "seedId": 124,
    "data": json.dumps(seed_data),
    "runtimeVars": json.dumps(runtime_vars),
}

resp = requests.post(url, json=data, headers=headers)
print(resp.json())

In [ ]:
from utils.tos import download_one_sample

download_one_sample("data/videos/raw/test/test1.mp4", "test_file/Ahrvn7IVgHk.mp4")

In [ ]:
# download from youtube
import yt_dlp
import os
from tqdm import tqdm
import json

resolution = "360"
destination_folder = f"data/videos/raw/{resolution}p/"
# 确保目标文件夹存在
os.makedirs(destination_folder, exist_ok=True)

with open("data/annotations/video_list_CZ.json", "r") as f:
    data = json.load(f)

for video in data:
    url = video["video_url"]
    pruned_url, video_id = parse_url(url)
    if pruned_url is None:
        video["path"] = None
    else:
        video["path"] = f"{destination_folder}{video_id}.mp4"
        video["video_url"] = pruned_url

with open("data/annotations/video_list_CZ_modified.json", "w") as f:
    json.dump(data, f, indent=4)

# 视频链接
urls = [
    "https://www.youtube.com/watch?v=PnvZZwlN2yk&ab_channel=KevinLangue",
]

# 配置下载选项
# options = {
#     # "format": f"bestvideo[height={resolution}]+bestaudio/best",
#     "format": f"bestvideo[ext=mp4][vcodec=h264][height={resolution}]+bestaudio[ext=m4a]/best[ext=mp4]",
#     "merge_output_format": "mp4",  # 确保最终输出为 MP4
#     # "outtmpl": destination_folder + "%(title)s.%(ext)s",
#     "postprocessors": [{"key": "FFmpegVideoConvertor", "preferedformat": "mp4"}],
#     # "quiet": True,
# }

# 创建下载器对象并下载
count = 0

for video in tqdm(data):
    if video["path"] is not None:
        options = {
            "format": f"bestvideo[ext=mp4][vcodec=h264][height={resolution}]+bestaudio[ext=m4a]/best[ext=mp4]",
            "merge_output_format": "mp4",  # 确保最终输出为 MP4
            "outtmpl": video["path"],
            "postprocessors": [
                {"key": "FFmpegVideoConvertor", "preferedformat": "mp4"}
            ],
            "quiet": True,
        }
        with yt_dlp.YoutubeDL(options) as ydl:
            try:
                # Update output template to use the specified path
                ydl.download([video["video_url"]])
                print(f"Successfully downloaded video to {video['path']}")
                count += 1
            except Exception as e:
                print(f"Error downloading {video['video_url']}: {str(e)}")

print(f"Total downloaded videos: {count}")